In [2]:
import cv2
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [6]:
# Initialize Cam
Cap = cv2.VideoCapture('Cam-6.mp4')


In [7]:
#Initialize Algorith for background subtraction
Algo = cv2.createBackgroundSubtractorMOG2(detectShadows=True)

In [8]:
line_Position = 550 
min_width_rect = 80
min_height_rect = 80

def center_pointer(x,y,w,h):
    x1 = int(w/2)
    y1 = int(h/2)
    cx = x+x1
    cy = y+y1
    return cx , cy
detect = []    
offset = 6
counter = 0
while True:
    ret , frame1 = Cap.read()
    grey = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
    bluring = cv2.GaussianBlur(grey,(3,3),5)
    apply_algo = Algo.apply(bluring)
    dilate = cv2.dilate(apply_algo,np.ones((5,5)))
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
    dilatada = cv2.morphologyEx(dilate,cv2.MORPH_CLOSE,kernel)
    dilatada = cv2.morphologyEx(dilatada,cv2.MORPH_CLOSE,kernel)
    counter_shape,h = cv2.findContours(dilatada,cv2.RETR_TREE , cv2.CHAIN_APPROX_SIMPLE)
    cv2.line(frame1,(25,line_Position),(1200,line_Position),(225,127,0),4)
    
    for (i,c) in enumerate(counter_shape):
        (x,y,w,h) = cv2.boundingRect(c)
        validator = (w >= min_width_rect) and (h >= min_height_rect)
        if not validator:
            continue
        cv2.rectangle(frame1,(x,y),(x+w,y+h),(0,225,0),3)  
        center = center_pointer(x,y,w,h)
        detect.append(center)
        cv2.circle(frame1,center,4,(0,0,225),-1)
        for (x,y) in detect:
            if y<(line_Position+offset) and y>(line_Position-offset):
                counter+=1
            cv2.line(frame1,(25,line_Position),(1200,line_Position),(225,127,0),4)
            detect.remove((x,y))
            #print('Vehichle_counter:'+str(counter))
    cv2.putText(frame1,'Vehichle_counter:'+str(counter),(450,70),cv2.FONT_HERSHEY_COMPLEX_SMALL,2,(0,0,225),5)      
                
    cv2.imshow('Test_Video_Original', frame1)

    if cv2.waitKey(1) == 13:
        break

cv2.destroyAllWindows()
Cap.release()

In [1]:
import cv2
import numpy as np

class Kordinat:
    def __init__(self,x,y):
        self.x=x
        self.y=y

class Sensor:
    def __init__(self,kordinat1,kordinat2,frame_weight,frame_lenght):
        self.kordinat1=kordinat1
        self.kordinat2=kordinat2
        self.frame_weight=frame_weight
        self.frame_lenght =frame_lenght
        self.mask=np.zeros((frame_weight,frame_lenght,1),np.uint8)*abs(self.kordinat2.y-self.kordinat1.y)
        self.full_mask_area=abs(self.kordinat2.x-self.kordinat1.x)
        cv2.rectangle(self.mask,(self.kordinat1.x,self.kordinat1.y),(self.kordinat2.x,self.kordinat2.y),(255),thickness=cv2.FILLED)
        self.stuation=False
        self.car_number_detected=0


Sensor1 = Sensor(Kordinat(1, 300), Kordinat(1080, 300), 500, 1080)
video=cv2.VideoCapture("Cam-6.mp4")
fgbg=cv2.createBackgroundSubtractorMOG2()
#fgbg=cv2.createBackgroundSubtractorMOG2()
kernel=np.ones((5,5),np.uint8)
font=cv2.FONT_HERSHEY_TRIPLEX
while (1):
    ret,frame=video.read()
    # resize frame
    cut_image=frame[100:600,100:1180]
    # make morphology for frame
    deleted_background=fgbg.apply(cut_image)
    opening_image=cv2.morphologyEx(deleted_background,cv2.MORPH_OPEN,kernel)
    ret,opening_image=cv2.threshold(opening_image,125,255,cv2.THRESH_BINARY)

    # detect moving anything
    cnts, _=cv2.findContours(opening_image,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
    result=cut_image.copy()

    zeros_image=np.zeros((cut_image.shape[0],cut_image.shape[1],1),np.uint8)

    # detect moving anything with loop
    for cnt in cnts:
        x,y,w,h=cv2.boundingRect(cnt)
        if (w>100 and h>100 ):
            cv2.rectangle(result,(x,y),(x+w,y+h),(255,0,0),thickness=2)
            cv2.rectangle(zeros_image,(x,y),(x+w,y+h),(255),thickness=cv2.FILLED)

    # detect whether there is car via bitwise_and
    mask1=np.zeros((zeros_image.shape[0],zeros_image.shape[1],1),np.uint8)
    mask_result=cv2.bitwise_or(zeros_image,zeros_image,mask=Sensor1.mask)
    white_cell_number=np.sum(mask_result==255)

    # detect to control whether car is passing under the red line sensor
    sensor_rate=white_cell_number/Sensor1.full_mask_area
    if sensor_rate>0:
        print(sensor_rate)

    # if car is passing under the red line sensor . red line sensor is yellow color.
    if (sensor_rate >= 1.8 and sensor_rate<2.9 and Sensor1.stuation == False):
        # draw the red line
        cv2.rectangle(result, (Sensor1.kordinat1.x, Sensor1.kordinat1.y), (Sensor1.kordinat2.x, Sensor1.kordinat2.y),
                      (0, 0, 255), thickness=cv2.FILLED)
        Sensor1.stuation = False
        Sensor1.car_number_detected += 2
    if (sensor_rate>=0.6 and  sensor_rate<1.8 and Sensor1.stuation==False):
        # draw the red line
        cv2.rectangle(result, (Sensor1.kordinat1.x, Sensor1.kordinat1.y), (Sensor1.kordinat2.x, Sensor1.kordinat2.y),
                      (0,255, 0,), thickness=cv2.FILLED)
        Sensor1.stuation = True
    elif (sensor_rate<0.2 and Sensor1.stuation==True) :
        # draw the red line
        cv2.rectangle(result, (Sensor1.kordinat1.x, Sensor1.kordinat1.y), (Sensor1.kordinat2.x, Sensor1.kordinat2.y),
                      (0, 0,255), thickness=cv2.FILLED)
        Sensor1.stuation = False
        Sensor1.car_number_detected+=1
    else :
        # draw the red line
        cv2.rectangle(result, (Sensor1.kordinat1.x, Sensor1.kordinat1.y), (Sensor1.kordinat2.x, Sensor1.kordinat2.y),
                      (0, 0, 255), thickness=cv2.FILLED)


    cv2.putText(result,str(Sensor1.car_number_detected),(Sensor1.kordinat1.x,Sensor1.kordinat1.y+60),font,2,(255,255,255))


    cv2.imshow("video", result)
    #cv2.imshow("mask_result", mask_result)
    #cv2.imshow("zeros_image", zeros_image)
    #cv2.imshow("opening_image", opening_image)

    if cv2.waitKey(1) == 13:
        break

video.release()
cv2.destroyAllWindows()

1.0
0.09545875810936053
0.0963855421686747
0.09545875810936053
0.0963855421686747
0.09823911028730306
0.09916589434661724
0.09731232622798888
0.10009267840593142
0.09731232622798888
0.09731232622798888
0.10009267840593142
0.09823911028730306
0.09545875810936053
0.09453197405004633
0.09453197405004633
0.09545875810936053
0.0963855421686747
0.0963855421686747
0.09823911028730306
0.1010194624652456
0.1102873030583874
0.11121408711770157
0.11214087117701575
0.11306765523632993
0.11677479147358666
0.11770157553290084
0.09545875810936053
0.09731232622798888
0.12974976830398516
0.13067655236329936
0.10287303058387395
0.10658016682113068
0.14365152919369786
0.11121408711770157
0.1380908248378128
0.1371640407784986
0.12048192771084337
0.12048192771084337
0.11677479147358666
0.1380908248378128
0.12511584800741427
0.13901760889712697
0.13994439295644115
0.14087117701575533
0.1417979610750695
0.14457831325301204
0.14921223354958293
0.15477293790546803
0.15755329008341057
0.1566265060240964
0.15384

0.1881371640407785
0.1974050046339203
0.19833178869323448
0.19833178869323448
0.19833178869323448
0.19833178869323448
0.19833178869323448
0.19833178869323448
0.19833178869323448
0.19833178869323448
0.20389249304911955
0.20389249304911955
0.20111214087117701
0.19925857275254866
0.211306765523633
0.211306765523633
0.211306765523633
0.22335495829471733
0.22335495829471733
0.22335495829471733
0.22335495829471733
0.22335495829471733
0.22706209453197404
0.22706209453197404
0.22706209453197404
0.2428174235403151
0.2428174235403151
0.22242817423540315
0.2567191844300278
0.2428174235403151
0.16682113067655235
0.16682113067655235
0.14550509731232622
0.13901760889712697
0.13901760889712697
0.1380908248378128
0.1417979610750695
0.14365152919369786
0.14457831325301204
0.14272474513438368
0.14272474513438368
0.12511584800741427
0.12511584800741427
0.12418906394810009
0.11770157553290084
0.11492122335495829
0.11399443929564411
0.1102873030583874
0.1102873030583874
0.11121408711770157
0.11121408711770

0.1872103799814643
0.18628359592215013
0.19555143651529194
0.2205746061167748
0.21408711770157554
0.20945319740500462
0.2020389249304912
0.19925857275254866
0.19647822057460612
0.19647822057460612
0.19647822057460612
0.20111214087117701
0.20018535681186284
0.20111214087117701
0.19925857275254866
0.19833178869323448
0.19833178869323448
0.19555143651529194
0.20389249304911955
0.19925857275254866
0.19555143651529194
0.18999073215940684
0.1881371640407785
0.18628359592215013
0.19184430027803523
0.19647822057460612
0.20296570898980537
0.1788693234476367
0.20296570898980537
0.20389249304911955
0.2057460611677479
0.20481927710843373
0.21223354958294718
0.21037998146431883
0.20759962928637626
0.20481927710843373
0.20481927710843373
0.21037998146431883
0.20389249304911955
0.19555143651529194
0.1881371640407785
0.19925857275254866
0.1974050046339203
0.1974050046339203
0.20296570898980537
0.211306765523633
0.211306765523633
0.20759962928637626
0.211306765523633
0.20759962928637626
0.2038924930491

0.5208526413345691
0.5152919369786839
0.5097312326227988
0.5032437442075997
0.4976830398517145
0.49212233549582945
0.4856348470806302
0.47914735866543096
0.4726598702502317
0.46617238183503246
0.45968489341983315
0.4522706209453197
0.4457831325301205
0.4392956441149212
0.432808155699722
0.42539388322520855
0.41797961075069506
0.41056533827618164
0.4031510658016682
0.3938832252085264
0.386468952734013
0.37812789620018533
0.3716404077849861
0.3632993512511585
0.35588507877664505
0.34754402224281744
0.3382761816496756
0.329935125115848
0.32252085264133457
0.3123262279888786
0.30954587581093607
0.2928637627432808
0.28174235403151066
0.2706209453197405
0.25857275254865614
0.24930491195551435
0.23540315106580167
0.23076923076923078
0.22706209453197404
0.21223354958294718
0.18628359592215013
0.17979610750695088
0.1686746987951807
0.14921223354958293
0.12882298424467098
0.12140871177015755
0.10750695088044486
1.0
1.0
0.4114921223354958
0.5690454124189064
0.7099165894346617
1.0
1.0
1.0
1.0
1.0
